In [ ]:
!pip install datasets
!pip install transformers
!pip install --upgrade transformers
!pip install einops
!pip install openai
!pip install evaluate
!pip install torch
!pip install joblib
!pip install tensorflow
!pip install scikit-learn
!pip install tf-keras
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
!pip install --upgrade google-api-python-client --ignore-installed
!pip install --upgrade google-cloud --ignore-installed
!pip install --upgrade google-cloud-vision --ignore-installed
!pip install google-cloud-aiplatform --ignore-installed

In [3]:
!pip install -q -U google-generativeai

In [4]:
from datasets import load_dataset, load_from_disk
import pandas as pd
import numpy as np
import random

from datasets import load_dataset
# from google.colab import auth
import openai
import os
from openai import OpenAI

import pandas as pd
# from google.colab import drive
import zipfile
import tempfile
import shutil

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
def load_and_extract_dataset(zip_path):
    # Create a temporary directory
    temp_dir = '/tmp/dataset_extracted'
    
    # Ensure the directory is clean
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)
    
    # Unzip the file to the temporary directory
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    # Load the dataset from the extracted directory
    dataset = load_from_disk(temp_dir)

    return dataset

In [6]:
zip_name = 'dataset.zip'
dataset = load_and_extract_dataset(zip_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 359
    })
})

In [7]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# auth.authenticate_user()

client = OpenAI()

In [9]:
# Function to load the dataset and pick K random entries
def pick_random_entries(K):
    # pos = []
    # neg = []
    # for i in range(len(train_dataset)):
    #   if train_dataset[i]['label'] == 0 and len(neg) <= 5:
    #     neg.append(train_dataset[i])
    #   if train_dataset[i]['label'] == 1 and len(neg) <= 5:
    #     pos.append(train_dataset[i])

    random_entries = random.sample(list(train_dataset), K)
    # Delete the useless stuff
    # for entry in random_entries:
    #   del entry["query"]
    #   del entry["user"]

    return random_entries

In [10]:
k = 10

random_entries = pick_random_entries(k)

for entry in random_entries:
    print(entry)

{'text': "2 glasses of wine and I'm not even sleepy. If that doesn't make me tired then I don't know what will ", 'label': 0}
{'text': "The longest security line I have ever seen! And I'm at the end if the line  - http://bkite.com/07AzK", 'label': 0}
{'text': '@ckillius no problem, i thought it would make it more fun and interactive ', 'label': 1}
{'text': 'says goodmorning people..  lakas ng ulan! http://plurk.com/p/yq9tp', 'label': 1}
{'text': '@midnightamatory Me too. :&quot;&gt; ', 'label': 1}
{'text': 'I could Tweet about him all night lon ', 'label': 1}
{'text': 'homework. because i have nothing better to do ', 'label': 0}
{'text': 'Tamagotchi Plus Color! I want one! Now those bring back memories ', 'label': 1}
{'text': 'Slight headache today-was it the cheap champagne, the homemade redcurrant wine or the copious amounts of other wines? Or all of the above ', 'label': 0}
{'text': 'http://twitpic.com/8d1tn - Bye bye long hair ', 'label': 0}


In [11]:
prompt = ""

for entry in random_entries:
  prompt += 'POSITIVE: "' if entry['label'] == 1 else 'NEGATIVE: "'
  prompt += entry["text"] + '"\n'
print(prompt)

NEGATIVE: "2 glasses of wine and I'm not even sleepy. If that doesn't make me tired then I don't know what will "
NEGATIVE: "The longest security line I have ever seen! And I'm at the end if the line  - http://bkite.com/07AzK"
POSITIVE: "@ckillius no problem, i thought it would make it more fun and interactive "
POSITIVE: "says goodmorning people..  lakas ng ulan! http://plurk.com/p/yq9tp"
POSITIVE: "@midnightamatory Me too. :&quot;&gt; "
POSITIVE: "I could Tweet about him all night lon "
NEGATIVE: "homework. because i have nothing better to do "
POSITIVE: "Tamagotchi Plus Color! I want one! Now those bring back memories "
NEGATIVE: "Slight headache today-was it the cheap champagne, the homemade redcurrant wine or the copious amounts of other wines? Or all of the above "
NEGATIVE: "http://twitpic.com/8d1tn - Bye bye long hair "



In [12]:
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import joblib

# Initialize BERT components
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\praha\anaconda3\envs\algoverse-env\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\praha\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

C:\Users\praha\anaconda3\envs\algoverse-env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
import openai

format = content3 = "Format the output as follows:\n POSITIVE: \Your positive comment here.\\n NEGATIVE: \Your negative comment here.\\n"
content1 = "You will be provided with several examples of social media comments along with whether the comment is of positive (POSITIVE) or negative (NEGATIVE) sentiment. Generate a positive and negative data entry that are each as indistinguishable as possible from the examples. Ensure the generated comments are realistic, contextually appropriate, and free of escape characters. Also ensure that the generated comments contain a few minor spelling or grammar mistakes. Make sure that the generated comments sound as human as possible and haven't been generated before. " + format
content2 = "You will be provided with several examples of social media comments along with whether the comment is of positive (POSITIVE) or negative (NEGATIVE) sentiment. Generate a positive and negative data entry that are each as indistinguishable as possible from the examples. " + format
content3 = "Make data similar to the social media examples that have been provided. Add punctuation, capitalization and spelling errors. " + format
content4 = "Make data similar to the social media examples that have been provided. Add frequent punctuation, frequent capitalization, and frequent spelling errors to make the text sound more human. but make sure that the errors are not complete gibberish. Feel free to add emojis and infrequent hashtags. Make sure that each response is distinctly different from the previous one. " + format
content5 = "Make data similar to the social media examples that have been provided. Add frequent punctuation, frequent capitalization, and frequent spelling errors to make the text sound more human. but make sure that the errors are not complete gibberish. Try to add information about current real world events. Feel free to add emojis and infrequent hashtags. Make sure that each response is distinctly different from the previous one. " + format
def generate_tweet_like_text(num_responses):

    responses = []
    for _ in range(num_responses):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": content4},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,  # Adjust for creativity
            top_p=0.9,        # Adjust for diversity
        )
        responses.append(response.choices[0].message.content)

    return responses

num_responses = 1

tweets = generate_tweet_like_text(num_responses)
print(tweets)

<>:3: SyntaxWarning: invalid escape sequence '\Y'
<>:3: SyntaxWarning: invalid escape sequence '\Y'
C:\Users\praha\AppData\Local\Temp\ipykernel_25692\2697959696.py:3: SyntaxWarning: invalid escape sequence '\Y'
  format = content3 = "Format the output as follows:\n POSITIVE: \Your positive comment here.\\n NEGATIVE: \Your negative comment here.\\n"


['POSITIVE: "OMG I just got a new puppy!!! 🐶💖 Can\'t wait to spoil him rotten!! #PuppyLove"  \nNEGATIVE: "Ughhh, why is my internet so slow today? 😩 Can\'t even stream my shows!! #Frustrating"  \n\nPOSITIVE: "Just finished an amazing book! 📚✨ Highly recommend it to everyone!! #MustRead"  \nNEGATIVE: "Why do I always forget my umbrella when it rains?? 😤☔️ I\'m soaked!"  \n\nPOSITIVE: "Just found the cutest little cafe in town! ☕️❤️ Can\'t wait to go back!! #CoffeeLover"  \nNEGATIVE: "Why is it so hard to find parking in this city?!! 😡 It\'s a nightmare!!"  \n\nPOSITIVE: "Feeling super productive today! Got so much done! 💪🎉 #Winning"  \nNEGATIVE: "Why does every single delivery take FOREVER?? 😩 I\'m hungry now!"  \n\nPOSITIVE: "Just got back from the most amazing trip! 🌍✈️ Can\'t wait to share pics! #TravelGoals"  \nNEGATIVE: "Seriously, can we talk about how bad this movie was? 😒 I was sooo bored!"  ']


In [14]:
# Example training data
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import pandas as pd

human_texts = ""
for i in random_entries:
  human_texts += i['text']
machine_texts = ""
for tweet in tweets:
    if tweet.strip() != "":
      machine_texts += tweet
# Labels
# Extract embeddings
text = human_texts + machine_texts
# X = get_bert_embeddings(human_texts + machine_texts)
# X = X.transpose()

labels = np.array([1] * len(human_texts) + [0] * len(machine_texts))
labels = np.random.permutation(labels)
labels = list(labels[0:len(text)])

df = pd.DataFrame({'text': text, 'label': labels})
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'])

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Tokenize the dataset

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

In [16]:
def format_labels(example):
    example['label'] = torch.tensor(example['label'], dtype=torch.long)
    return example

train_dataset = train_dataset.map(format_labels)
test_dataset = test_dataset.map(format_labels)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

ArrowInvalid: Could not convert 0 with type numpy.int64: did not recognize Python value type when inferring an Arrow data type

In [47]:
# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=3,
    weight_decay=0.03,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Fine-tune the model
trainer.train()
# Save the tokenizer and model
tokenizer.save_pretrained('bert_tokenizer')
model.save_pretrained('bert_model')

Epoch,Training Loss,Validation Loss
1,No log,0.539847


In [40]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

training_epochs = 10
score = training_epochs
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert_tokenizer')
model = BertForSequenceClassification.from_pretrained('bert_model')

def evaluate_comment(comment):
    global score
    inputs = tokenizer(comment, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probs, dim=1).item()
    if prediction == 1:
        return "Human"
    else:
        score -= 1
        return "Machine"

# Example evaluation
for i in range(training_epochs):
    new_comment = generate_tweet_like_text(1)
    print("(" + str(i+1) + "/" + str(training_epochs) + ")", "Evaluation:", evaluate_comment(new_comment))
print("Training accuracy:", score/training_epochs)

(1/10) Evaluation: Machine
(2/10) Evaluation: Machine
(3/10) Evaluation: Machine
(4/10) Evaluation: Machine
(5/10) Evaluation: Machine
(6/10) Evaluation: Machine
(7/10) Evaluation: Machine
(8/10) Evaluation: Machine
(9/10) Evaluation: Machine
(10/10) Evaluation: Machine
Training accuracy: 0.0
